# SupplyChain class

In [1]:
import os
os.chdir('/Users/aciullo')

In [2]:
import numpy as np
import pandas as pd
from climada.hazard import TCTracks, TropCyclone, Centroids
from climada.entity import LitPop
from climada.entity import ImpactFuncSet, ImpfTropCyclone
from climada_petals.engine import SupplyChain

This tutorial shows how to use the `SupplyChain` class of CLIMADA. This class allows assessing indirect impacts via Input-Ouput modeling. Before diving into this class, it is highly recommended the user first familiarizes herself with the `Exposures`, `Hazard` and `Impact` classes.

## 1. Load Multi-Regional Input-Output Tables (MRIOT) data.

At first, one needs to load Input Output data. `SupplyChain` has a function to download and read multi-regional input-output tables (MRIOT) from the 2016 release of WIOD project (www.wiod.org). Yearly WIOT tables are available for the period 2000-2014. A table is automatically downloaded the first time it is used.

In [3]:
supplychain = SupplyChain()
supplychain.read_wiod16(year=2013)

898kKB [00:27, 32.1kKB/s]                           


Let's now look at what data are now loaded into `SupplyChain`, i.e. modelled countries, sectors and IO data structure.

In [4]:
supplychain.mriot_reg_names

array(['AUS', 'AUT', 'BEL', 'BGR', 'BRA', 'CAN', 'CHE', 'CHN', 'CYP',
       'CZE', 'DEU', 'DNK', 'ESP', 'EST', 'FIN', 'FRA', 'GBR', 'GRC',
       'HRV', 'HUN', 'IDN', 'IND', 'IRL', 'ITA', 'JPN', 'KOR', 'LTU',
       'LUX', 'LVA', 'MEX', 'MLT', 'NLD', 'NOR', 'POL', 'PRT', 'ROU',
       'RUS', 'SVK', 'SVN', 'SWE', 'TUR', 'TWN', 'USA', 'ROW'],
      dtype=object)

There are 43 countries plus 1. The additional "country" refers to all countries not explicitly modeled which are aggregated into a Rest of World (ROW) "country".

In [5]:
supplychain.sectors

array(['Crop and animal production, hunting and related service activities',
       'Forestry and logging', 'Fishing and aquaculture',
       'Mining and quarrying',
       'Manufacture of food products, beverages and tobacco products',
       'Manufacture of textiles, wearing apparel and leather products',
       'Manufacture of wood and of products of wood and cork, except furniture; manufacture of articles of straw and plaiting materials',
       'Manufacture of paper and paper products',
       'Printing and reproduction of recorded media',
       'Manufacture of coke and refined petroleum products ',
       'Manufacture of chemicals and chemical products ',
       'Manufacture of basic pharmaceutical products and pharmaceutical preparations',
       'Manufacture of rubber and plastic products',
       'Manufacture of other non-metallic mineral products',
       'Manufacture of basic metals',
       'Manufacture of fabricated metal products, except machinery and equipment',
       

In [6]:
print(supplychain.sectors.shape)

(56,)


There are 56 economic sectors. These sectors can also be grouped into higher-level sectors. For instance, in the aftermath we will model the service sector which will include the following sectors:

In [7]:
supplychain.sectors[range(26,56)]

array(['Construction',
       'Wholesale and retail trade and repair of motor vehicles and motorcycles',
       'Wholesale trade, except of motor vehicles and motorcycles',
       'Retail trade, except of motor vehicles and motorcycles',
       'Land transport and transport via pipelines', 'Water transport',
       'Air transport',
       'Warehousing and support activities for transportation',
       'Postal and courier activities',
       'Accommodation and food service activities',
       'Publishing activities',
       'Motion picture, video and television programme production, sound recording and music publishing activities; programming and broadcasting activities',
       'Telecommunications',
       'Computer programming, consultancy and related activities; information service activities',
       'Financial service activities, except insurance and pension funding',
       'Insurance, reinsurance and pension funding, except compulsory social security',
       'Activities auxiliar

Weather to aggregate sectors into main sectors and how to do it is up to the user, according to the application of interest and data availability. Default settings are available in CLIMADA based on the built-in datasets. These will be introduced below when calculating direct damages.

In [8]:
supplychain.mriot_data

array([[12040.172354214412, 101.9164143282377, 208.76741897905276, ...,
        7.9861927246568385, 0, 1.0916326583937792e-06],
       [86.283642969608, 203.7632366724908, 0.1934409618983819, ...,
        0.02361017526391285, 0, 2.909262924680758e-08],
       [19.789153310985746, 0.001052851734315676, 19.387062673105213,
        ..., 0.023757592716657012, 0, 2.804571243540667e-09],
       ...,
       [2.3321663653078577, 0.011569979643092579, 0.13354713249770248,
        ..., 16541.17685820414, 0, 0.8495958305743502],
       [0.047169644901188544, 3.5647133751547286e-06,
        0.0001487510381775775, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=object)

In [9]:
supplychain.mriot_data.shape

(2464, 2464)

The MRIO table is a squared matrix with columns (and rows) equal to the number of economic sectors times the number of modeled countries, i.e. 56x44 = 2464. Each column (row) reports input (output) data of *all* sectors of *a given* country untill all countries are reported. The total production from all subsectors of all countries is:

In [ ]:
supplychain.total_prod

In [ ]:
print(supplychain.total_prod.shape)

The following dict allows accessing mriot data of single countries:

In [ ]:
supplychain.reg_pos

For example, focusing on Switzerland, to find output data from all swiss sectors one can do:

In [ ]:
supplychain.mriot_data[supplychain.reg_pos['CHE']]

Similarly, one can find the total production of all Swiss sectors:

In [ ]:
supplychain.total_prod[supplychain.reg_pos['CHE']]

## 2. Define Hazard, Exposure and Vulnerability

Let's now define hazard, exposure and vulnerability. This is handled via the related CLIMADA classes. In this tutorial we use `LitPop` for exposure and `TropCyclone` for hazard. We will focus on the impact of tropical cyclones affecting the Philippines, Taiwan, Vietnam and Japan in 2012 and 2013. Japan and Taiwan are modeled explicitely by the MRIO table, while the Philippines and Vietnam are modeled as Rest of World, they are thus aggregated into a single country.

In [ ]:
countries = ['PHL', 'TWN', 'VNM', 'JPN']
exp_lp = LitPop()
exp_lp.set_country(countries, res_arcsec=150)
exp_lp.set_geometry_points()

In [ ]:
exp_lp.plot_hexbin(pop_name=False)

In [ ]:
tc_tracks=TCTracks.from_ibtracs_netcdf(year_range=(2012,2013), basin='WP')

In [ ]:
tc_tracks.plot()

In [ ]:
centr = Centroids.from_lat_lon(exp_lp.gdf.latitude.values, exp_lp.gdf.longitude.values)

In [ ]:
tc_cyclone = TropCyclone()
tc_cyclone.set_from_tracks(tracks=tc_tracks, centroids=centr)

In [ ]:
tc_cyclone.plot_intensity(event=0)

In [ ]:
impf_tc = ImpfTropCyclone.from_emanuel_usa()

# add the impact function to an Impact function set
impf_set = ImpactFuncSet()
impf_set.append(impf_tc)
impf_set.check()

In [ ]:
[haz_type] = impf_set.get_hazard_types()
[haz_id] = impf_set.get_ids()[haz_type]

# Exposures: rename column and assign id
exp_lp.gdf.rename(columns={"impf_": "impf_" + haz_type}, inplace=True)
exp_lp.gdf['impf_' + haz_type] = haz_id
exp_lp.check()

## 3. Calculate direct, indirect and total impact per sector and country

Let's now calculate direct, indirect and total impacts. For the **direct impact**, `SupplyChain` requires as inputs `Hazard`, `Exposures` and `ImpactFuncSet`. In addition, one may want to specify `selected_subsec`, which allows the user to either define her own aggregation of sectors by providing a list with the positions of the sectors to aggregate or to use built-in sectors aggregations passing a string being either `service`, `manufacturing`, `agriculture` or `mining`.

For this tutorial, we will model the service sector, as this sector's exposure can reasonably be modelled via nighlights and population data, i.e. via `LitPop`.

### 3.1 Direct impact

In [ ]:
supplychain.calc_sector_direct_impact(tc_cyclone, exp_lp, impf_set, selected_subsec='service')

Let's see what new attributes the class has got now.

In [ ]:
supplychain.direct_impact

In [ ]:
supplychain.direct_impact.shape

All impact matrixes (also those below) provide impacts aggregated over years. They have a number of rows equal to the years being modeled (2 years this time, i.e. 2012-2013) and columns equal to the number of countries times the number of sectors.
, i.e. 2464 (see also above).

In [ ]:
supplychain.direct_aai_agg

In [ ]:
supplychain.direct_aai_agg.shape

The annual aggregated impact (aai) matrixes provide yearly average impact. They are row vectors with columns equal to the number of countries times the number of sectors, i.e. 2464.

Info for a given country can be accessed as done below:

In [ ]:
supplychain.direct_aai_agg[supplychain.reg_pos['CHE']]

with e.g. CHE, we obviously get zeros, as we are modeling direct impacts in east Asia.

In [ ]:
supplychain.direct_aai_agg[supplychain.reg_pos['JPN']]

for e.g. Japan we instead have direct damages. In order to get all positions of countries undergoing direct damages, one can access the following list:

In [ ]:
supplychain.reg_dir_imp #note we have two ROW for PHL and VNM

and do the following:

In [ ]:
all_pos = [y for cntry in np.unique(supplychain.reg_dir_imp) for y in supplychain.reg_pos[cntry]]

In [ ]:
print(supplychain.direct_impact.sum(), supplychain.direct_impact[:, all_pos].sum())

In [ ]:
print(supplychain.direct_aai_agg.sum(), supplychain.direct_aai_agg[all_pos].sum())

i.e., the matrix has non-zero values only at positions corresponding to the modelled countries.

### 3.2 Indirect impact

For the **indirect impact**, one can choose the IO modeling approach between Leontief, Ghosh and EEIOA. References are provided below:

In [ ]:
supplychain.calc_indirect_impact?

Let's calculate indirect impacts according to the Ghosh method:

In [ ]:
supplychain.calc_indirect_impact(io_approach='ghosh')

The class now has the indirect impact matrix and vector, with structure equal to those introduced for the direct impact:

In [ ]:
supplychain.indirect_impact

In [ ]:
supplychain.indirect_impact.shape

In [ ]:
supplychain.indirect_aai_agg

If we now check damages for e.g. Switzerland:

In [ ]:
supplychain.indirect_aai_agg[supplychain.reg_pos['CHE']]

there are non-zero values, as CH undergoes indirect impacts due to events happening in east Asia.

We can also visualize coefficients, inverse matrix and risk matrix of the selected IO approach:

In [ ]:
supplychain.io_data

### 3.3 Total impact

Finally, let's calculate **total impacts**, as the sum of both direct and indirect. Therefore, the impact matrixes have the same structure as the direct and indirect matrixes.

In [ ]:
supplychain.calc_total_impact()

In [ ]:
supplychain.total_impact

In [ ]:
supplychain.total_aai_agg

Finally, one can for example visualize total annual average impacts to all Japanese (direct plus indirect) and Swiss (only direct) subsector after TC in East Asia:

In [ ]:
df_imp = pd.DataFrame(data=np.vstack([supplychain.total_aai_agg[supplychain.reg_pos['CHE']],
                                      supplychain.total_aai_agg[supplychain.reg_pos['JPN']]]), 
                      columns=supplychain.sectors,
                      index=['CHE', 'JPN'])

In [ ]:
df_imp #in M USD